In [2]:
from PIL import Image
from torchvision import transforms
from utils import load_model
from torchvision.models import resnet18
import torch
import os

PATH_TO_EVAL = 'evaluation/'
EVAL_IMAGES = os.listdir(PATH_TO_EVAL)

In [4]:
filename = 'models/resnet18_weights_best_acc.tar'  # path to the downloaded pre-trained model
use_gpu = False  # set to True if you want to load weights on the GPU
model = resnet18(num_classes=1081)  # 1081 classes in Pl@ntNet-300K

load_model(model, filename=filename, use_gpu=use_gpu)

29

In [5]:
type(model)

torchvision.models.resnet.ResNet

In [6]:
# get shape of the model
for name, param in model.named_parameters():
    print(f'{name:42} {str(param.shape):30} {param.requires_grad}')

conv1.weight                               torch.Size([64, 3, 7, 7])      True
bn1.weight                                 torch.Size([64])               True
bn1.bias                                   torch.Size([64])               True
layer1.0.conv1.weight                      torch.Size([64, 64, 3, 3])     True
layer1.0.bn1.weight                        torch.Size([64])               True
layer1.0.bn1.bias                          torch.Size([64])               True
layer1.0.conv2.weight                      torch.Size([64, 64, 3, 3])     True
layer1.0.bn2.weight                        torch.Size([64])               True
layer1.0.bn2.bias                          torch.Size([64])               True
layer1.1.conv1.weight                      torch.Size([64, 64, 3, 3])     True
layer1.1.bn1.weight                        torch.Size([64])               True
layer1.1.bn1.bias                          torch.Size([64])               True
layer1.1.conv2.weight                      torch.Siz

In [5]:
# load monstera image
monstera = Image.open('monstera.jpg')

In [6]:
# classify image
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])
monstera_tensor = transform(monstera).unsqueeze(0)

In [7]:
# Set the model to evaluation mode
model.eval()

# Perform inference
with torch.no_grad():
    output = model(monstera_tensor)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Get the predicted class and the confidence score
_, predicted_class = torch.max(output, 1)
confidence_score = probabilities[predicted_class].item()


In [8]:
_, predicted_class, confidence_score

(tensor([11.6517]), tensor([285]), 0.5636008977890015)

In [8]:
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])    


In [18]:
def predict(path_to_image, transform, model):
    image = Image.open(path_to_image)
    image_tensor = transform(image).unsqueeze(0)

    # Set the model to evaluation mode
    model.eval()

    # Perform inference
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # Get the predicted class and the confidence score
    _, predicted_class = torch.max(output, 1)
    confidence_score = probabilities[predicted_class].item()

    return _, predicted_class, confidence_score

# predict all images in the evaluation folder
for image in EVAL_IMAGES:
    path_to_image = PATH_TO_EVAL + image
    _, predicted_class, confidence_score = predict(path_to_image, transform, model)
    print(f'Image: {image:40} | Predicted class: {str(predicted_class):15} | Confidence score: {confidence_score:.2f}')


Image: annie-spratt-8mqOw4DBBSg-unsplash.jpg    | Predicted class: tensor([334])   | Confidence score: 0.91
Image: annie-spratt-hX_hf2lPpUU-unsplash.jpg    | Predicted class: tensor([322])   | Confidence score: 0.41
Image: chris-lee-70l1tDAI6rM-unsplash.jpg       | Predicted class: tensor([858])   | Confidence score: 0.14
Image: feey-7d7OR-RvufU-unsplash.jpg            | Predicted class: tensor([702])   | Confidence score: 0.71
Image: igor-son-FV_PxCqgtwc-unsplash.jpg        | Predicted class: tensor([92])    | Confidence score: 0.30
Image: lauren-mancke-DpphPG9ENsI-unsplash.jpg   | Predicted class: tensor([434])   | Confidence score: 0.37
Image: linh-le-Ebwp2-6BG8E-unsplash.jpg         | Predicted class: tensor([322])   | Confidence score: 0.46
Image: monstera.jpg                             | Predicted class: tensor([285])   | Confidence score: 0.56
Image: ren-ran-bBiuSdck8tU-unsplash.jpg         | Predicted class: tensor([602])   | Confidence score: 0.19
Image: teemu-paananen-OOE4xA

In [9]:
# load plantnet300k_metadata.json
import json
with open('plantnet300k_metadata.json') as f:
    metadata = json.load(f)
print(1)

1


In [11]:
type(metadata)

dict

In [18]:
# print 20 random keys and their values
import random
random.seed(42)
random_keys = random.sample(list(metadata.keys()), 20)
for key in random_keys:
    print(f'{key:10} {metadata[key]}')


a80a1fb1383acdec6f9073553c2e0645ea2d55da {'species_id': '1358689', 'obs_id': '1004630654', 'organ': 'fruit', 'is_shareable': True, 'v1_id': '', 'author': 'Kieron Dunne', 'license': 'cc-by-sa', 'split': 'train'}
eae259d1df04404150ba79219bf84a1597cb3254 {'species_id': '1385937', 'obs_id': '1007897628', 'organ': 'leaf', 'is_shareable': True, 'v1_id': '', 'author': 'Petrika Mirna', 'license': 'cc-by-sa', 'split': 'train'}
511b857913c00ef1fee795077de154f077e4c1a9 {'species_id': '1359616', 'obs_id': '1005326602', 'organ': 'fruit', 'is_shareable': True, 'v1_id': '', 'author': 'gerard dumont', 'license': 'cc-by-sa', 'split': 'train'}
01e217a0cfa6620d27e4fc3c43ce73949ab37287 {'species_id': '1359517', 'obs_id': '1008500291', 'organ': 'flower', 'is_shareable': True, 'v1_id': '', 'author': 'Alfredo', 'license': 'cc-by-sa', 'split': 'val'}
04fc5ebbc933427550a71196cb49410fe8becb55 {'species_id': '1384497', 'obs_id': '1004762218', 'organ': 'fruit', 'is_shareable': True, 'v1_id': '', 'author': "that's

In [28]:
# get list of all unique obs_id in metadata
species_ids = set()
for key in metadata.keys():
    species_ids.add(metadata[key]['species_id'])

In [29]:
len(species_ids)

1081

In [30]:
# sort obs_ids and print first 10
species_ids = sorted(species_ids)
species_ids[:10]

['1355868',
 '1355920',
 '1355932',
 '1355936',
 '1355937',
 '1355955',
 '1355959',
 '1355961',
 '1355978',
 '1355990']